# Stallion Motors — Forecasting (6-month)

This notebook demonstrates two approaches for forecasting revenue: Prophet and seasonal ARIMA (pmdarima).

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet  # pip install prophet

monthly = pd.read_csv('stallion_monthly_agg.csv', parse_dates=['ds'])
monthly = monthly[['ds','Revenue']]
monthly.head()

In [ ]:
# Fit Prophet model
m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
m.fit(monthly.rename(columns={'ds':'ds','Revenue':'y'}))
future = m.make_future_dataframe(periods=6, freq='M')
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(10)

## ARIMA fallback (pmdarima)
If Prophet isn't available, you can use pmdarima for automatic SARIMA selection.

In [ ]:
from pmdarima import auto_arima
series = monthly.set_index('ds')['Revenue']
model = auto_arima(series, seasonal=True, m=12, trace=True, error_action='ignore')
model.summary()

In [ ]:
# Forecast with ARIMA
n_periods = 6
arima_forecast = model.predict(n_periods=n_periods)
last_date = series.index.max()
future_index = pd.date_range(last_date + pd.offsets.MonthBegin(1), periods=n_periods, freq='M')
arima_df = pd.DataFrame({'ds': future_index, 'yhat': arima_forecast})
arima_df

## Save forecasts to CSV for dashboard

In [ ]:
forecast[['ds','yhat','yhat_lower','yhat_upper']].to_csv('stallion_prophet_forecast.csv', index=False)
arima_df.to_csv('stallion_arima_forecast.csv', index=False)
print('Saved forecasts')